In [1]:
# GitHub Link: https://github.com/Lasata-T/Final_Netflix_Project.git

In [2]:
# imports
import csv
import pandas as pd
import os
import datetime
import pymongo
import pprint

In [3]:
# pip install --upgrade pip

In [4]:
# pip install tflearn

In [5]:
# pip install tensorflow

In [6]:
# !pip install tensorflow --upgrade --force-reinstall

In [7]:
# start chatbot

import nltk 
# import discord
nltk.download('punkt')

from nltk import word_tokenize,sent_tokenize

from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
# read more on the steamer https://towardsdatascience.com/stemming-lemmatization-what-ba782b7c0bd8
import numpy as np 
import tflearn
import tensorflow as tf
import random
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/maniktuladhar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
with open("new_intents.json") as file: 
    data = json.load(file)

In [9]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
        if intent["tag"] not in labels:
            labels.append(intent["tag"])


    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []
    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
               bag.append(1)
            else:
              bag.append(0)
    
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1
        
        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output = np.array(output)
    
    with open("data.pickle","wb") as f:
        pickle.dump((words, labels, training, output), f)

In [10]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 4999  | total loss: 0.95398 | time: 0.008s
| Adam | epoch: 1000 | loss: 0.95398 - acc: 0.9451 -- iter: 32/33
Training Step: 5000  | total loss: 0.85919 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.85919 - acc: 0.9506 -- iter: 33/33
--
INFO:tensorflow:/Users/maniktuladhar/Desktop/NEW_Netflix_Project/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [12]:
try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

INFO:tensorflow:Restoring parameters from /Users/maniktuladhar/Desktop/NEW_Netflix_Project/model.tflearn


In [13]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return np.array(bag)

In [14]:
mongo_db = pymongo.MongoClient('mongodb://localhost:27017/')

In [15]:
# loading in relevant CSVs (show data only)
shows = pd.read_csv('Best Shows Netflix.csv')

# only using SCORE column to determine show ranking, so don't need NUMBER_OF_VOTES column
shows.drop('NUMBER_OF_VOTES', axis = 1)

,index,TITLE,RELEASE_YEAR,SCORE,DURATION,NUMBER_OF_SEASONS,MAIN_GENRE,MAIN_PRODUCTION
0,0,Breaking Bad,2008,9.5,48,5,drama,US
1,1,Avatar: The Last Airbender,2005,9.3,24,3,scifi,US
2,2,Our Planet,2019,9.3,50,1,documentary,GB
3,3,Kota Factory,2019,9.3,42,2,drama,IN
4,4,The Last Dance,2020,9.1,50,1,documentary,US
...,...,...,...,...,...,...,...,...
241,241,Evil Genius,2018,7.5,48,1,crime,US
242,242,13 Reasons Why,2017,7.5,58,4,drama,US
243,243,Lupin,2021,7.5,46,3,crime,FR
244,244,All of Us Are Dead,2022,7.5,61,1,action,KR


In [16]:
shows_year = pd.read_csv('Best Show by Year Netflix.csv')
# only using MAIN_PRODUCTION column from general df, so don't need it here 
shows_year.drop('MAIN_PRODUCTION', axis = 1)

,index,TITLE,RELEASE_YEAR,SCORE,NUMBER_OF_SEASONS,MAIN_GENRE
0,0,Monty Python's Flying Circus,1969,8.8,4,comedy
1,1,Knight Rider,1982,6.9,4,action
2,2,Seinfeld,1989,8.9,9,comedy
3,3,Star Trek: Deep Space Nine,1993,8.1,7,scifi
4,4,Neon Genesis Evangelion,1995,8.5,1,scifi
5,5,Stargate SG-1,1997,8.4,10,scifi
6,6,Cowboy Bebop,1998,8.9,1,western
7,7,One Piece,1999,8.8,21,action
8,8,Gilmore Girls,2000,8.2,8,comedy
9,9,Trailer Park Boys,2001,8.6,12,comedy


In [17]:
# creating a MongoDB
db = mongo_db['shows']
mongo_db.list_database_names()

['admin', 'config', 'local', 'shows', 'shows_year']

In [18]:
# creating a Mongo collection
shows_collection = db.collection.insert_many(shows.to_dict('records'))

In [19]:
# checking Mongo collection lists
db.list_collection_names()

['collection']

In [20]:
# creating a MongoDB for shows_year
db2 = mongo_db['shows_year']
mongo_db.list_database_names()

['admin', 'config', 'local', 'shows', 'shows_year']

In [21]:
# creating a Mongo collection for shows_year
shows_yrs_collection = db2.collection2.insert_many(shows.to_dict('records'))

In [22]:
# checking updated Mongo collection lists
db2.list_collection_names()

['collection', 'collection2']

In [137]:
def chat():
    print("Start talking with the bot! (type quit to stop)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        result = model.predict([bag_of_words(inp, words)])[0]
        result_index = np.argmax(result)
        tag = labels[result_index]
        
        if result[result_index] > 0.7:
            for tg in data["intents"]:
                if inp.lower() == "help":
                    print('\n')
                    print("Here are the types of questions you can ask me: \n")
                    print("1: What are the top three highest rated shows of 2020?")
                    print("2: How many seasons did the top show of 2022 have?")
                    print("3: For 2018, what shows had an episode duration of 30 minutes or less?")
                    print("4: For the highest rated show in 2005, what was its genre?")
                    print("5: Where was the show with the highest rating of 2021 produced?")
                    print("6: In 2008, what was the name of the highest rated show?")
                    print("7: What are all the shows that are rated 9.0 and higher?")
                    print("8: What are all the shows made in Japan after 2000?")
                    print("9: What are all the shows that start with the letter W?")
                    print("10: What shows are 4 seasons long?")
                    print('\n')
                    break
            
                if tag == 'question 1':
                    filtered = db['collection'].find(filter = {'RELEASE_YEAR': 2020}, limit = 3)

                    top_3_shows = list(filtered)
                    
                    print('\n')
                    for three in top_3_shows:
                        print(three['TITLE'])
                    print('\n')
                    
                    break
                    
                elif tag == 'question 2':
                    filtered = db2['collection2'].find(filter = {'RELEASE_YEAR': 2022}, limit = 1)

                    num_seasons = list(filtered)
                    
                    print('\n')
                    print(num_seasons[0]['NUMBER_OF_SEASONS'])
                    print('\n')
                    
                    break
                    
                elif tag == 'question 3':
                    filtered = db['collection'].find(filter = {'RELEASE_YEAR': 2018, 'DURATION': {'$lte': 30}}, limit = 10)
                    shows_under_30 = list(filtered)
                    
                    print('\n')
                    for ten in shows_under_30:
                        print(ten['TITLE'])
                    print('\n')
                    
                    break
                    
                elif tag == 'question 4':
                    filtered = db2['collection2'].find(filter = {'RELEASE_YEAR': 2005}, limit = 1)

                    genre = list(filtered)
                    
                    print('\n')
                    print(genre[0]['MAIN_GENRE'])
                    print('\n')
                    
                    break
                    
                elif tag == 'question 5':
                    filtered = db['collection'].find(filter = {'RELEASE_YEAR': 2021}, limit = 1)

                    country = list(filtered)
                    
                    print('\n')
                    print(country[0]['MAIN_PRODUCTION'])
                    print('\n')
                    
                    break
                    
                elif tag == 'question 6':
                    print('\n')
                    pprint.pprint(db['collection'].find_one()['TITLE'])
                    print('\n')
                    
                elif tag == 'question 7':
                    filtered = db['collection'].find(filter = {'SCORE': {'$gte': 9.0}}, limit = 9)
                    shows_over_9 = list(filtered)
                    
                    print('\n')
                    for nine in shows_over_9:
                        print(nine['TITLE'])
                    print('\n')
                    
                    break
                    
                elif tag == 'question 8':
                    print('\n')
                    print('q8')
                    print('\n')
                    
                    break
                    
                elif tag == 'question 9':
                    filtered = db['collection'].find(filter = {'TITLE': {'$regex': '^W'}}, limit = 4)
                    W_shows = list(filtered)
                    
                    print('\n')
                    for w in W_shows:
                        print(w['TITLE'])
                    print('\n')
                    
                    break
                    
                elif tag == 'question 10':
                    filtered = db2['collection2'].find(filter = {'NUMBER_OF_SEASONS': 4}, limit = 4)

                    four_seasons = list(filtered)
                    
                    print('\n')
                    for four in four_seasons:
                        print(four['TITLE'])
                    print('\n')
                    
                    break

        else:
            print('\n')
            print("I didn't get that. Can you please choose from this list of questions? \n")
            print("1: What are the top three highest rated shows of 2020?")
            print("2: How many seasons did the top show of 2022 have?")
            print("3: For 2018, what shows had an episode duration of 30 minutes or less?")
            print("4: For the highest rated show in 2005, what was its genre?")
            print("5: Where was the show with the highest rating of 2021 produced?")
            print("6: In 2008, what was the name of the highest rated show?")
            print("7: What are all the shows that are rated 9.0 and higher?")
            print("8: What are all the shows made in Japan after 2000?")
            print("9: What are all the shows that start with the letter W?")
            print("10: What shows are 4 seasons long?")
            print('\n')
chat()

Start talking with the bot! (type quit to stop)
You: help


I didn't get that. Can you please choose from this list of questions? 

1: What are the top three highest rated shows of 2020?
2: How many seasons did the top show of 2022 have?
3: For 2018, what shows had an episode duration of 30 minutes or less?
4: For the highest rated show in 2005, what was its genre?
5: Where was the show with the highest rating of 2021 produced?
6: In 2008, what was the name of the highest rated show?
7: What are all the shows that are rated 9.0 and higher?
8: What are all the shows made in Japan after 2000?
9: What are all the shows that start with the letter W?
10: What shows are 4 seasons long?


You: For 2018, what shows had an episode duration of 30 minutes or less?


Hilda
Derry Girls
Violet Evergarden
The Dragon Prince
The Kominsky Method
Rascal Does Not Dream of Bunny Girl Senpai
On My Block
She-Ra and the Princesses of Power
Devilman Crybaby
Everything Sucks!


You: quit
